In [26]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import image_uris
from sagemaker.session import s3_input, Session

In [14]:
bucket_name = 'projectvanila'
my_region = boto3.session.Session().region_name
print(my_region)

eu-north-1


In [16]:
s3 = boto3.resource('s3', region_name='eu-north-1')

try:
    if s3.meta.client.meta.region_name == 'eu-north-1':
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': 'eu-north-1'}
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)


S3 bucket created successfully


In [17]:
#setting path where my model is supposed to save.
prefix = 'xgboost-builtin'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://projectvanila/xgboost-builtin/output


In [18]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [20]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [23]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
     

In [24]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')
     

In [27]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
region = boto3.Session().region_name
container = image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.0-1'
)

In [28]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [31]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

estimator = Estimator(
    image_uri=container,
    hyperparameters=hyperparameters,
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,  # GB
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600
)


In [32]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-06-25-18-38-22-932


2025-06-25 18:38:25 Starting - Starting the training job...
2025-06-25 18:38:56 Downloading - Downloading input data...
2025-06-25 18:39:11 Downloading - Downloading the training image...
2025-06-25 18:39:47 Training - Training image download completed. Training in progress...[2025-06-25 18:40:02.435 ip-10-0-85-6.eu-north-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.

In [36]:
xgb_predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'  # change to a supported type in your region
)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-06-25-19-00-49-528
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-06-25-19-00-49-528
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-06-25-19-00-49-528


------!

In [37]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

ImportError: cannot import name 'csv_serializer' from 'sagemaker.predictor' (/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/predictor.py)

In [ ]:
predictions_array

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
     